In [ ]:
# 1. Google 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 2. 필요한 라이브러리 설치
!pip install pydub

In [ ]:
# 3. 라이브러리 임포트
import os
import random
from pydub import AudioSegment
from openpyxl import Workbook

In [ ]:
def mix_audio_files(audio_files, output_path, silence_thresh=-50.0, target_length_ms=5000):
    """
    여러 오디오 파일을 믹싱하여 하나의 오디오 파일로 출력합니다.
    """
    first_audio = AudioSegment.from_file(audio_files[0])
    second_audio = AudioSegment.from_file(audio_files[1])
    mixed_audio = first_audio.overlay(second_audio)

    final_audio = mixed_audio[:target_length_ms]
    final_audio.export(output_path, format="wav")
    #print(f"{os.path.basename(output_path)} saved: mix {first_audio} and {second_audio}")

In [ ]:
import os
import random
from openpyxl import Workbook
import shutil

def create_mixed_audio_files(base_path, output_folder, num_files, spreadsheet_path="file_paths.xlsx"):
    """
    human 폴더에서 랜덤으로 두 개의 파일을 선택하여 믹싱하고,
    'train', 'test', 'valid' 폴더에 지정된 비율로 분류하여 저장합니다.
    """
    # 경로 설정
    # human_base = os.path.join(base_path, 'human')

    # 출력 폴더 및 하위 폴더 생성
    subsets = ['train', 'test', 'valid']
    subset_counts = {'train': 400, 'test': 60, 'valid': 140}

    for subset in subsets:
        # os.makedirs(os.path.join(output_folder, subset), exist_ok=True)
        # os.makedirs(f'/content/drive/MyDrive/Dataset/Human_Human/s1/{subset}', exist_ok=True)
        # os.makedirs(f'/content/drive/MyDrive/Dataset/Human_Human/s2/{subset}', exist_ok=True)
        os.makedirs(os.path.join(output_folder, subset, 's1'), exist_ok=True)
        os.makedirs(os.path.join(output_folder, subset, 's2'), exist_ok=True)
        os.makedirs(os.path.join(output_folder, subset, 'mixed_data'), exist_ok=True)

    # 스프레드시트 초기화
    wb = Workbook()
    ws = wb.active
    ws.append(["index", "human-file1", "human-file2", "subset"])  # 첫 번째 row 설정

    # 파일 생성 및 저장
    file_index = {'train': 0, 'test': 0, 'valid': 0}  # 각 subset의 생성된 파일 수
    for i in range(num_files):
        # 랜덤으로 human 파일 두 개 선택
        human_files = [f for f in os.listdir(base_path) if os.path.isfile(os.path.join(base_path, f))]
        selected_human_files = random.sample(human_files, 2)
        selected_human_path1 = os.path.join(base_path, selected_human_files[0])
        selected_human_path2 = os.path.join(base_path, selected_human_files[1])

        # 선택된 파일 이름 출력
        print(f"Human_Human_{i + 1}.wav saved: mix human1={selected_human_files[0]}, human2={selected_human_files[1]}")

        # 파일 믹싱 및 저장
        # 현재 subset 결정
        for subset, count in subset_counts.items():
            if file_index[subset] < count:
                current_subset = subset
                break

        # Mixed 파일 저장
        # subset_folder = os.path.join(output_folder, current_subset)
        # output_path = os.path.join(subset_folder, f"Human_Human_{i + 1}.wav")
        # mix_audio_files([selected_human_path1, selected_human_path2], output_path=output_path)
        mixed_folder = os.path.join(output_folder, current_subset, 'mixed_data')
        mixed_output_path = os.path.join(mixed_folder, f"Human_Human_{i + 1}.wav")
        mix_audio_files([selected_human_path1, selected_human_path2], output_path=mixed_output_path)

        # 원본 파일 복사 및 저장
        # s1_path = f'/content/drive/MyDrive/Dataset/Human_Human/s1/{current_subset}/Human_Human_{i + 1}.wav'
        # s2_path = f'/content/drive/MyDrive/Dataset/Human_Human/s2/{current_subset}/Human_Human_{i + 1}.wav'
        s1_folder = os.path.join(output_folder, current_subset, 's1')
        s2_folder = os.path.join(output_folder, current_subset, 's2')
        s1_path = os.path.join(s1_folder, f"Human_Human_{i + 1}.wav")
        s2_path = os.path.join(s2_folder, f"Human_Human_{i + 1}.wav")
        shutil.copy(selected_human_path1, s1_path)
        shutil.copy(selected_human_path2, s2_path)

        # 한 줄씩 스프레드시트에 기록 (인덱스와 파일 경로, subset 정보)
        ws.append([i + 1, selected_human_files[0], selected_human_files[1], current_subset])

        # 해당 subset의 파일 개수 증가
        file_index[current_subset] += 1

    # 엑셀 저장
    wb.save(spreadsheet_path)
    print(f"Spreadsheet saved at {spreadsheet_path}")
    return wb

In [ ]:
base_path = '/content/drive/MyDrive/Dataset/RawData/human'  # 기본 경로

output_folder = '/content/drive/MyDrive/Dataset/mk_1800_dataset'  # 출력 폴더
num_files = 600  # 생성할 믹싱 파일 수 #16000

wb = create_mixed_audio_files(base_path, output_folder, num_files=num_files)

Human_Human_1.wav saved: mix human1=002c0e0k.wav_normalized.wav, human2=431o051n.wav_normalized.wav
Human_Human_2.wav saved: mix human1=p226_normalized.wav, human2=20sc010e.wav_normalized.wav
Human_Human_3.wav saved: mix human1=400o072i.wav_normalized.wav, human2=002o0o17.wav_normalized.wav
Human_Human_4.wav saved: mix human1=400o010c.wav_normalized.wav, human2=MDPS0_normalized.wav
Human_Human_5.wav saved: mix human1=430o0d0q.wav_normalized.wav, human2=443c040h.wav_normalized.wav
Human_Human_6.wav saved: mix human1=FSAK0_normalized.wav, human2=423e090i.wav_normalized.wav
Human_Human_7.wav saved: mix human1=444e090y.wav_normalized.wav, human2=443e0912.wav_normalized.wav
Human_Human_8.wav saved: mix human1=MDLR1_normalized.wav, human2=409c0203.wav_normalized.wav
Human_Human_9.wav saved: mix human1=400c0m22.wav_normalized.wav, human2=400o0h2d.wav_normalized.wav
Human_Human_10.wav saved: mix human1=432c0c13.wav_normalized.wav, human2=p376_normalized.wav
Human_Human_11.wav saved: mix human1